# Example 2: Computing p-values with DTransFusion

This notebook demonstrates how to use the `PPL_SI` package to compute selective inference p-values for the DTransFusion algorithm.

In [1]:
import numpy as np
import sys
sys.path.append('..')

from ppl_si import generate_synthetic_data, DTF_SI, DTF_SI_randj

## 1. Generate Synthetic Data

In [2]:
np.random.seed(42)

p = 300
num_sh = 10
num_inv = 5
K = 5
n_list = [100, 100, 100, 100, 50]
true_beta_sh = 0.3
Gamma = 0.1
itc = 0.5

X_list, Y_list, true_betaK, Sigma_list = generate_synthetic_data(
    p=p,
    num_sh=num_sh,
    num_inv=num_inv,
    K=K,
    n_list=n_list,
    true_beta_sh=true_beta_sh,
    Gamma=Gamma,
    itc=itc
)

print(f"Number of tasks: {K}")
print(f"Feature dimension: {p}")
print(f"Target sample size: {n_list[-1]}")

Number of tasks: 5
Feature dimension: 300
Target sample size: 50


## 2. Set Parameters

In [3]:
lambda_k_list = [np.sqrt(2 * np.log(p) / n_list[k]) for k in range(K)]
lambda_0 = 1.5 * np.sqrt(np.log(p) / sum(n_list))
lambda_tilde = 1.5 * np.sqrt(np.log(p) / n_list[-1])
qk_weights = [0.1 * np.sqrt(n_list[k] / sum(n_list)) for k in range(K - 1)]
z_min = -20
z_max = 20

print(f"lambda_0: {lambda_0:.4f}")
print(f"lambda_tilde: {lambda_tilde:.4f}")
print(f"qk_weights: {[f'{w:.4f}' for w in qk_weights]}")

lambda_0: 0.1689
lambda_tilde: 0.5066
qk_weights: ['0.0471', '0.0471', '0.0471', '0.0471']


## 3. Compute p-values for All Selected Features

In [4]:
p_values = DTF_SI(
    X_list=X_list,
    Y_list=Y_list,
    lambda_k_list=lambda_k_list,
    lambda_0=lambda_0,
    lambda_tilde=lambda_tilde,
    qk_weights=qk_weights,
    Sigma_list=Sigma_list,
    z_min=z_min,
    z_max=z_max
)

if p_values is not None:
    print(f"\nNumber of selected features: {len(p_values)}")
    print("\nFeature index and p-values:")
    for j, p_val in p_values:
        is_true_positive = true_betaK[j] != 0.0
        marker = "✓" if is_true_positive else "✗"
        print(f"Feature {j}: p-value = {p_val:.4f} {marker}")
else:
    print("No features selected")


Number of selected features: 6

Feature index and p-values:
Feature 0: p-value = 0.6127 ✓
Feature 1: p-value = 0.0271 ✓
Feature 4: p-value = 0.0170 ✓
Feature 91: p-value = 0.0964 ✗
Feature 112: p-value = 0.3202 ✗
Feature 210: p-value = 0.8632 ✗


## 4. Compute p-value for a Random Selected Feature

In [5]:
p_value_rand = DTF_SI_randj(
    X_list=X_list,
    Y_list=Y_list,
    lambda_k_list=lambda_k_list,
    lambda_0=lambda_0,
    lambda_tilde=lambda_tilde,
    qk_weights=qk_weights,
    Sigma_list=Sigma_list,
    z_min=z_min,
    z_max=z_max
)

if p_value_rand is not None:
    print(f"Random feature p-value: {p_value_rand:.4f}")
else:
    print("No features selected")

Random feature p-value: 0.6127


## 5. Analysis

In [6]:
if p_values is not None:
    alpha = 0.05
    significant_features = [(j, p) for j, p in p_values if p <= alpha]
    
    true_positives = sum(1 for j, _ in p_values if true_betaK[j] != 0.0)
    false_positives = len(p_values) - true_positives
    
    print(f"\nSignificance level: {alpha}")
    print(f"Number of selected features: {len(p_values)}")
    print(f"  - True positives: {true_positives}")
    print(f"  - False positives: {false_positives}")
    print(f"Number of significant features: {len(significant_features)}")
    
    if len(significant_features) > 0:
        print("\nSignificant features:")
        for j, p_val in significant_features:
            is_true_positive = true_betaK[j] != 0.0
            marker = "✓" if is_true_positive else "✗"
            print(f"Feature {j}: p-value = {p_val:.4f} {marker}")


Significance level: 0.05
Number of selected features: 6
  - True positives: 3
  - False positives: 3
Number of significant features: 2

Significant features:
Feature 1: p-value = 0.0271 ✓
Feature 4: p-value = 0.0170 ✓
